In [8]:
import pandas as pd

In [9]:
df = pd.read_csv('movies_new.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19925 entries, 0 to 19924
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0.1  19925 non-null  int64 
 1   Unnamed: 0    19925 non-null  int64 
 2   show_id       19925 non-null  object
 3   type          19925 non-null  object
 4   title         19925 non-null  object
 5   director      14736 non-null  object
 6   cast          17677 non-null  object
 7   country       10569 non-null  object
 8   date_added    10399 non-null  object
 9   release_year  19925 non-null  int64 
 10  rating        19581 non-null  object
 11  duration      19922 non-null  object
 12  listed_in     19925 non-null  object
 13  description   19925 non-null  object
dtypes: int64(3), object(11)
memory usage: 2.1+ MB


In [10]:
data = df[df['country'].isnull()]
data1 = df[~df['country'].isnull()]

In [4]:
from functools import lru_cache
from imdb import IMDb

@lru_cache(maxsize=9000)  # Cache up to 9000 movie titles



def get_movie_country(movie_title):
    try:
        ia = IMDb()
        movies = ia.search_movie(movie_title)
        if movies:
            ids = movies[0].movieID
            series = ia.get_movie(ids)
            countries = series.get('countries')
            if countries:
                return countries[0]
    except Exception:
        # Return None in case of any error
        return None

movie_title = "The Dark Knight"
filming_country = get_movie_country(movie_title)

if filming_country:
    print(f"The movie '{movie_title}' was filmed in {filming_country}.")
else:
    print(f"Could not find filming country information for '{movie_title}'.")

The movie 'The Dark Knight' was filmed in United States.


In [24]:
import concurrent.futures
import pandas as pd
import time
from functools import lru_cache
from imdb import IMDb

@lru_cache(maxsize=9000)  # Cache up to 9000 movie titles
def get_movie_country(movie_title):
    try:
        ia = IMDb()
        movies = ia.search_movie(movie_title)
        if movies:
            ids = movies[0].movieID
            series = ia.get_movie(ids)
            countries = series.get('countries')
            if countries:
                return countries[0]
    except Exception:
        # Return None in case of any error
        return None

def process_data_batch(data_batch):
    data_batch['country'] = data_batch['title'].apply(get_movie_country)
    print(f"Processed {len(data_batch)} records successfully.")

# Split the data into batches of size 100
batch_size = 10
data_batches = [data[i:i+batch_size] for i in range(0, len(data), batch_size)]

# Define a function to process a data batch using multiple threads
def process_batch_threaded(data_batch):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []
        for idx, row in data_batch.iterrows():
            title = row['title']
            future = executor.submit(get_movie_country, title)
            futures.append((future, idx))

        for future, idx in futures:
            country = future.result()
            data_batch.at[idx, 'country'] = country

    print(f"Processed {len(data_batch)} records successfully.")

# Process the data batches
start_time = time.time()
for i, data_batch in enumerate(data_batches):
    process_batch_threaded(data_batch)

    # Optionally, you can add a delay between batches to avoid overwhelming the API server
    time.sleep(1)

    # Print progress after processing each batch
    if (i + 1) % 10 == 0:  # Adjust the value (10) to control the frequency of progress updates
        elapsed_time = time.time() - start_time
        print(f"Processed {i + 1} batches out of {len(data_batches)}. Elapsed time: {elapsed_time:.2f} seconds.")

# Merge the processed data batches back into a single DataFrame
processed_data = pd.concat(data_batches)

Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 batches out of 936. Elapsed time: 141.76 seconds.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 20 batches out of 936. Elapsed time: 289.91 seconds.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.

Processed 200 batches out of 936. Elapsed time: 3136.50 seconds.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 210 batches out of 936. Elapsed time: 3294.24 seconds.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 220 batches out of 936. Elapsed time: 3452.77 seconds.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.


2023-05-30 11:27:13,528 CRITICAL [imdbpy] C:\ProgramData\anaconda3\lib\site-packages\imdb\_exceptions.py:32: IMDbDataAccessError exception raised; args: ({'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/title/tt26403733/plotsummary', 'proxy': '', 'exception type': 'IOError', 'original exception': RemoteDisconnected('Remote end closed connection without response')},); kwds: {}
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\lib\site-packages\imdb\parser\http\__init__.py", line 233, in retrieve_unicode
    response = uopener.open(url)
  File "C:\ProgramData\anaconda3\lib\urllib\request.py", line 525, in open
    response = meth(req, response)
  File "C:\ProgramData\anaconda3\lib\urllib\request.py", line 634, in http_response
    response = self.parent.error(
  File "C:\ProgramData\anaconda3\lib\urllib\request.py", line 557, in error
    result = self._call_chain(*args)
  File "C:\ProgramData\anaconda3\lib\urllib\request.py", line 496, in _call_chain
  

2023-05-30 11:27:13,580 CRITICAL [imdbpy] C:\ProgramData\anaconda3\lib\site-packages\imdb\__init__.py:850: caught an exception retrieving or parsing "plot" info set for mopID "10214826" (accessSystem: http)
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\lib\urllib\request.py", line 1348, in do_open
    h.request(req.get_method(), req.selector, req.data, headers,
  File "C:\ProgramData\anaconda3\lib\http\client.py", line 1282, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "C:\ProgramData\anaconda3\lib\http\client.py", line 1328, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "C:\ProgramData\anaconda3\lib\http\client.py", line 1277, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "C:\ProgramData\anaconda3\lib\http\client.py", line 1037, in _send_output
    self.send(msg)
  File "C:\ProgramData\anaconda3\lib\http\client.py", line 975, in send
    self.conne

2023-05-30 11:27:20,585 CRITICAL [imdbpy] C:\ProgramData\anaconda3\lib\site-packages\imdb\_exceptions.py:32: IMDbDataAccessError exception raised; args: ({'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/title/tt6118426/reference', 'proxy': '', 'exception type': 'IOError', 'original exception': TimeoutError('The read operation timed out')},); kwds: {}
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\lib\site-packages\imdb\parser\http\__init__.py", line 233, in retrieve_unicode
    response = uopener.open(url)
  File "C:\ProgramData\anaconda3\lib\urllib\request.py", line 519, in open
    response = self._open(req, data)
  File "C:\ProgramData\anaconda3\lib\urllib\request.py", line 536, in _open
    result = self._call_chain(self.handle_open, protocol, protocol +
  File "C:\ProgramData\anaconda3\lib\urllib\request.py", line 496, in _call_chain
    result = func(*args)
  File "C:\ProgramData\anaconda3\lib\urllib\request.py", line 1391, in https_open
    r

...

Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 620 batches out of 936. Elapsed time: 9805.20 seconds.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 630 batches out of 936. Elapsed time: 9959.31 seconds.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 640 batches out of 9

2023-05-30 13:42:56,383 CRITICAL [imdbpy] C:\ProgramData\anaconda3\lib\site-packages\imdb\_exceptions.py:32: IMDbDataAccessError exception raised; args: ({'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/title/tt10640760/reference', 'proxy': '', 'exception type': 'IOError', 'original exception': TimeoutError('The read operation timed out')},); kwds: {}
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\lib\site-packages\imdb\parser\http\__init__.py", line 233, in retrieve_unicode
    response = uopener.open(url)
  File "C:\ProgramData\anaconda3\lib\urllib\request.py", line 519, in open
    response = self._open(req, data)
  File "C:\ProgramData\anaconda3\lib\urllib\request.py", line 536, in _open
    result = self._call_chain(self.handle_open, protocol, protocol +
  File "C:\ProgramData\anaconda3\lib\urllib\request.py", line 496, in _call_chain
    result = func(*args)
  File "C:\ProgramData\anaconda3\lib\urllib\request.py", line 1391, in https_open
    

Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 790 batches out of 936. Elapsed time: 12512.33 seconds.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 800 batches out of 936. Elapsed time: 12682.04 seconds.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records success

2023-05-30 14:13:19,304 CRITICAL [imdbpy] C:\ProgramData\anaconda3\lib\site-packages\imdb\_exceptions.py:32: IMDbDataAccessError exception raised; args: ({'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/find/?q=Maz+Jobrani%3A+Immigrant&s=tt', 'proxy': '', 'exception type': 'IOError', 'original exception': TimeoutError('The read operation timed out')},); kwds: {}
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\lib\site-packages\imdb\parser\http\__init__.py", line 233, in retrieve_unicode
    response = uopener.open(url)
  File "C:\ProgramData\anaconda3\lib\urllib\request.py", line 519, in open
    response = self._open(req, data)
  File "C:\ProgramData\anaconda3\lib\urllib\request.py", line 536, in _open
    result = self._call_chain(self.handle_open, protocol, protocol +
  File "C:\ProgramData\anaconda3\lib\urllib\request.py", line 496, in _call_chain
    result = func(*args)
  File "C:\ProgramData\anaconda3\lib\urllib\request.py", line 1391, in http

Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 910 batches out of 936. Elapsed time: 14357.71 seconds.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 920 batches out of 936. Elapsed time: 14510.47 seconds.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records successfully.
Processed 10 records success

In [25]:
processed_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9356 entries, 9879 to 19924
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0.1  9356 non-null   int64 
 1   Unnamed: 0    9356 non-null   int64 
 2   show_id       9356 non-null   object
 3   type          9356 non-null   object
 4   title         9356 non-null   object
 5   director      7340 non-null   object
 6   cast          8035 non-null   object
 7   country       8119 non-null   object
 8   date_added    993 non-null    object
 9   release_year  9356 non-null   int64 
 10  rating        9040 non-null   object
 11  duration      9356 non-null   object
 12  listed_in     9356 non-null   object
 13  description   9356 non-null   object
dtypes: int64(3), object(11)
memory usage: 1.1+ MB


In [30]:
df1 = pd.concat([data1,processed_data])
df1.drop(['Unnamed: 0.1', 'Unnamed: 0'], axis=1, inplace=True)

In [31]:
df1

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,The Grand Seduction,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",Canada,"March 30, 2021",2014,NaN,113 min,"Comedy, Drama",A small fishing village must procure a local d...
1,s2,Movie,Take Care Good Night,Girish Joshi,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",India,"March 30, 2021",2018,13+,110 min,"Drama, International",A Metro Family decides to fight a Cyber Crimin...
2,s3,Movie,Secrets of Deception,Josh Webber,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",United States,"March 30, 2021",2017,NaN,74 min,"Action, Drama, Suspense",After a man discovers his wife is cheating on ...
3,s4,Movie,Pink: Staying True,Sonia Anderson,"Interviews with: Pink, Adele, Beyoncé, Britney...",United States,"March 30, 2021",2014,NaN,69 min,Documentary,"Pink breaks the mold once again, bringing her ..."
4,s5,Movie,Monster Maker,Giles Foster,"Harry Dean Stanton, Kieran O'Brien, George Cos...",United Kingdom,"March 30, 2021",1989,NaN,45 min,"Drama, Fantasy",Teenage Matt Banting wants to work with a famo...
...,...,...,...,...,...,...,...,...,...,...,...,...
19920,s1137,Movie,Planet of the Birds,NaN,Rodd Houston,United States,"November 12, 2019",2018,TV-PG,45 min,"Animals & Nature, Documentary, Family","Tough and versatile, birds live in almost ever..."
19921,s1205,TV Show,Spider-Man (Shorts),NaN,NaN,None,"November 12, 2019",2016,TV-Y7,1 Season,"Action-Adventure, Animation, Kids",Peter uses science to investigate his spider b...
19922,s1211,TV Show,Star Wars Rebels (Shorts),NaN,NaN,United States,"November 12, 2019",2014,TV-Y7,1 Season,"Action-Adventure, Animation, Kids",It is a dark time in the galaxy as the Empire ...
19923,s1260,TV Show,The Book of Once Upon a Time,NaN,Kate Mulvany,Australia,"November 12, 2019",2014,TV-Y,4 Seasons,"Animation, Anthology, Kids",Classic Disney tales are brought to life in a ...


In [32]:
df1.to_csv('movie_proccessed_data2.csv',index=False)